<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/infer_ort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime
!wget https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
!wget https://github.com/onnx/models/blob/1008b6e9450ab632538390e38e0ab28b3f207fbc/vision/classification/squeezenet/model/squeezenet1.1-7.onnx?raw=true
!mv squeezenet1.1-7.onnx?raw=true squeezenet1.1-7.onnx

In [ ]:
import numpy as np
from PIL import Image

import onnxruntime as rt
sess_options = rt.SessionOptions()
sess_options.enable_profiling = True

model_path = "./squeezenet1.1-7.onnx"
input_file="./jet.jpeg"
EP_list = ['CPUExecutionProvider']
session = rt.InferenceSession(model_path, providers=EP_list, sess_options=sess_options)


# get the name of the first input of the model
input_name = session.get_inputs()[0].name
input_details  = session.get_inputs()
output_details = session.get_outputs()

#Pre-Process input
image    = Image.open(input_file).resize((224, 224))
img_data = np.array(image).transpose(2, 0, 1).astype('float32')
mean_vec   = np.array([0.485, 0.456, 0.406])
stddev_vec = np.array([0.229, 0.224, 0.225])
norm_img_data = np.zeros(img_data.shape).astype('float32')
for i in range(img_data.shape[0]):
    norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
input_data = norm_img_data.reshape(1, 3, 224, 224)

#Run inference session
raw_result = session.run([], {input_name: input_data})

#Post Process Result
res = np.array(raw_result[0]).reshape(-1)
print(res.argsort()[-5:][::-1])

prof_file = session.end_profiling()
print(prof_file)
